In [104]:
import pandas as pd
import torch
import torchvision
from torch.utils.data import Dataset
import torch.nn as nn
from torchvision import transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import random_split

In [105]:
df = pd.read_csv(r'C:\Users\Lenovo\Documents\kmnist.csv')
X = df.drop('label',axis=1)
X = X.values.reshape(-1,28,28)
X = X/255.0
y = df.label

In [114]:
X.shape

(36495, 28, 28)

In [106]:
class CustomImageDataset(Dataset):
    def __init__(self, X,y):
        self.img = torch.tensor(X)
        self.img_labels = torch.LongTensor(y)

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):

        return self.img[idx].reshape(1,28,28), self.img_labels[idx]

In [107]:
dataset = CustomImageDataset(X,y)

In [108]:
train,test = random_split(dataset,[10949, 25546])

In [109]:
train_loader = torch.utils.data.DataLoader(dataset=train,batch_size=100)
test_loader = torch.utils.data.DataLoader(dataset=test,
                                           batch_size=100)


In [110]:
class LeNet5(nn.Module):
    def __init__(self, num_classes):
        super(LeNet5, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=5, stride=1,padding=1)
        self.conv_layer2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=5, stride=1,padding=0)
        self.conv_layer3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1,padding=0)
        self.conv_layer4 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5, stride=1,padding=0)
        self.pooling_layer1 = nn.MaxPool2d(kernel_size=2)

        self.linear_layer1 = nn.Linear(in_features=32, out_features=64)
        self.linear_layer2 = nn.Linear(in_features=64, out_features=num_classes)

        self.relu = nn.ReLU()
        self.softmax = nn.Softmax()

    def forward(self, inputs):
        print(inputs.shape,inputs.type())
        output_1 = self.relu(self.conv_layer1(inputs))
        output_2 = self.pooling_layer1(output_1)
        output_3 = self.relu(self.conv_layer2(output_2))
        output_4 = self.pooling_layer2(output_3)
        output_5 = self.tanh(self.conv_layer3(output_4))
        output_6 = torch.flatten(output_5, 1)

        output_7 = self.relu(self.linear_layer1(output_6))
        output = self.linear_layer2(output_7)

        return self.softmax(output)

In [111]:
model = LeNet5(14)
print(model)

LeNet5(
  (conv_layer1): Conv2d(1, 8, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (conv_layer2): Conv2d(8, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv_layer3): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv_layer4): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
  (pooling_layer1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear_layer1): Linear(in_features=32, out_features=64, bias=True)
  (linear_layer2): Linear(in_features=84, out_features=14, bias=True)
  (relu): ReLU()
  (softmax): Softmax(dim=None)
)


In [112]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
total_step = len(train_loader)
for epoch in range(5):
    for i, (X_test, y_train) in enumerate(train_loader):
        outputs = model(X_test)
        loss = criterion(outputs, y_train)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, 5, i+1, total_step, loss.item()))

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy: {} %'.format(100 * correct / total))